<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/week4/Deep_Neural_Network_Application_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이미지 분류를 위한 심층 신경망 만들어보기 #

이 과제를 마무리하면, 우리는 4주차 마지막, 다시 말해 이번 코스의 마지막 과제를 끝내게 될 것입니다.

이번 과제에서는 지난 4주차 첫 번째 과제에서 구현했던 함수들을 사용해서 고양이 이미지를 분류하는 심층 신경망을 만들 것입니다. 2주차의 로지스틱 회귀 구현에 비해 정확도가 향상될 것을 기대해보며, 시작해봅시다.

**이번 과제가 끝나고 여러분은:**
- 지도 학습 방식으로 훈련된 심층 신경망을 개발할 수 있게 됩니다.

이제 시작해봅시다!

## 1. 패키지 다운로드 ##

과제를 수행하는데 필요한 모든 패키지를 다운로드합니다.

- [numpy](www.numpy.org)는 파이썬의 가장 기본적인 과학/수학 연산 관련 패키지입니다.
- [h5py](http://www.h5py.org)는 .h5에 저장된 데이터셋과 상호작용하기 위한 패키지입니다.
- [matplotlib](http://matplotlib.org)은 파이썬 환경에서 그래프를 그릴 수 있도록 해주는 유명한 라이브러리입니다.
- [PIL](http://www.pythonware.com/products/pil/) 과 [scipy](https://www.scipy.org/)는 모든 모델의 학습이 끝난 후, 자기가 가지고 있는 샘플 사진으로 모델을 테스트하는데 사용됩니다.
- `dnn_app_utils` 는 4주차 첫 번째 과제인 *Building your Deep Neural Network: Step by Step* 에서 구현했던 함수들을 모은 패키지입니다.
- `np.random.seed(1)` 은 랜덤 함수가 일관된 결과를 만들도록 합니다. 채점을 위해 시드를 변경하지 마세요.



In [ ]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from dnn_app_utils_v3 import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2. 데이터셋 ##

이번 과제에서는 지난 2주차 과제 *Logistic Regression as a Neural Network* 에서 사용했었던 고양이 데이터를 그대로 사용합니다. 2주차 과제에서 만들었던 고양이 분류 모델의 최종 정확도는 `70%`였습니다. 이번에 새롭게 만들 모델은 그보다 더 높은 정확도를 가지길 기대해 봅니다.

**문제 정의**: 주어진 데이터 셋 `data.h5` 에 대하여,
- 훈련 세트 `m_train`은 고양이가 맞음(1), 고양이가 아님(1) 두 종류로 라벨링된 이미지 데이터 묶음입니다.
- 테스트 세트 `m_test`는 훈련 세트와 마찬가지로 라벨링된 이미지 데이터 묶음입니다.
- 각 이미지는 (num_px, num_px, 3) 의 형태를 하고 있습니다. 여기서 3은 색상 채널(RGB)를 의미합니다.

In [ ]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

아래 코드는 데이터셋의 이미지가 어떻게 생겼는지 보여줍니다. 데이터 셋의 인덱스를 자유롭게 바꾸고 재 시도하여 다른 이미지를 확인해보세요.

In [ ]:
# Example of a picture
index = 10
plt.imshow(train_x_orig[index])
print ("y = " + str(train_y[0,index]) + ". It's a " + classes[train_y[0,index]].decode("utf-8") +  " picture.")

In [ ]:
# Explore your dataset 
m_train = train_x_orig.shape[0]
num_px = train_x_orig.shape[1]
m_test = test_x_orig.shape[0]

print ("Number of training examples: " + str(m_train))
print ("Number of testing examples: " + str(m_test))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_x_orig shape: " + str(train_x_orig.shape))
print ("train_y shape: " + str(train_y.shape))
print ("test_x_orig shape: " + str(test_x_orig.shape))
print ("test_y shape: " + str(test_y.shape))

늘 그랬듯이, 이미지를 신경망에 투입하기 적합한 사이즈로 재배치(reshape) 하고 표준화(standardize) 합시다. 바로 아래 코드 블록을 실행시켜 보세요.

<img src="arts/imvectorkiank.png" style="width:450px;height:300px;">

<caption><center> <u>Figure 1</u>: Image to vector conversion. <br> </center></caption>

In [ ]:
# Reshape the training and test examples 
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

# Standardize data to have feature values between 0 and 1.
train_x = train_x_flatten/255.
test_x = test_x_flatten/255.

print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))


결과값 $12,288$은 $64 \times 64 \times S$(= 재배치된 이미지 벡터 하나의 크기) 입니다.

## 3. 모델 구성하기 ##

데이터 셋에 좀 익숙해졌나요? 이제는 고양이 이미지를 분류하는 심층 신경망 모델을 만들 차례입니다.

이 과제에선 두 개의 다른 모델을 만들 것입니다.
- 2-layer 신경망
- L-layer 심층 신경망

$L$ 값을 달리 하면 모델의 성능에 어떤 변화가 있는지를 비교하면서 이번 과제를 진행합니다.

아래 두 모델의 신경망 구조를 확인해보세요

### 3-1. 2-layer 신경망 ###

<img src="arts/2layerNN_kiank.png" style="width:650px;height:400px;">
<caption><center> <u>Figure 2</u>: 2-layer neural network. <br> The model can be summarized as: <b>INPUT -> LINEAR -> RELU -> LINEAR -> SIGMOID -> OUTPUT.</b> </center></caption>

<br>

<u>그림 2에 대한 구체적인 설명</u>:
- (64, 64, 3) 형태의 입력 이미지는 (12288, 1) 형태의 벡터로 변형됩니다. 이 벡터가 신경망의 최종 입력값으로 들어갑니다.
- 이 입력값을 전치한 행렬 $[x_0,x_1,...,x_{12287}]^T$ 을 가중치 행렬 $W^{[1]}$과 곱합니다.
- 이후 bias 값 $b^{[1]}$를 더하고, 최종 값에 `relu` 함수를 취해서 최종 활성화값을 담은 벡터 $[a_0^{[1]}, a_1^{[1]},..., a_{n^{[1]}-1}^{[1]}]^T$ 를 얻을 수 있습니다.
- 이 과정을 1번 더 반복합니다.
- 그 결과값에 $W^{[2]}$ 벡터를 곱하고, bias $b^{[2]}$를 더합니다.
- 최종 계산 값에 `sigmoid` 함수를 취합니다. 그 결과가 0.5보다 크면 고양이로 분류합니다.

<br>

### 3-2. L-layer 심층 신경망 ###

L개의 층을 가진 심층 신경망을 위 그림처럼 직접 표현하는 것은 어려운 일입니다. 그래서, 아래 그림처럼 일부 간소화된 신경망을 표현하는 그림을 준비했습니다.

<img src="arts/LlayerNN_kiank.png" style="width:650px;height:400px;">
<caption><center> <u>Figure 3</u>: L-layer neural network. <br> The model can be summarized as: <b>[LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID</b></center></caption>

</br>

<u>그림 3에 대한 구체적인 설명</u>:
- (64, 64, 3) 형태의 입력 이미지는 (12288, 1) 형태의 벡터로 변형됩니다. 이 벡터가 신경망의 최종 입력값으로 들어갑니다.
- 이 입력값을 전치한 행렬 $[x_0,x_1,...,x_{12287}]^T$ 을 가중치 행렬 $W^{[1]}$과 곱하고, bias $b^{[l]}$와 더합니다. 이 결과를 linear unit이라 부릅니다.
- 다음으로, 이 linear unit에 `relu` 함수를 취합니다. 일련의 과정은 모델의 구조에 따라 매 $l$번 층의 파라미터 $(W^{[l]}, b^{[l]})$에 대해서 반복적으로 이루어집니다.
- 최종적인 linear unit에 `sigmoid` 함수를 취해, 그 겨로가값이 0.5보다 크면 고양이로 분류합니다.

<br>

### 3-3. 보다 일반적인 방법론 ###

기본적으로 모델을 구축하기 위해선 아래의 딥 러닝 방법론을 따르는 것이 좋습니다.
1. 파라미터 초기화 / 하이퍼 파라미터 정의
2. 사전에 정해진 반복 횟수만큼 반복하기
  a. forward propagation
  b. 비용 함수 계산
  c. backward propagation
  d. 파라미터 업데이트 (경사 하강법 사용)
3. 훈련된 파라미터들을 사용해서 예측하기.

이제 이 두 모델을 구현해보겠습니다.


## 4. 2-layer 신경망 구현해보기 ##

**연습 문제** 지난 과제에서 만들었던 helper 함수들을 사용해서 2개의 층을 가지는 신경망을 만들어봅시다. 2-layer의 구조는 *LINEAR -> RELU -> LINEAR -> SIGMOID* 과 같습니다. 지난번에 구현한 함수의 대략적인 구조는 아래와 같습니다.
  ```python
  def initialize_parameters(n_x, n_h, n_y):
      ...
      return parameters 
  def linear_activation_forward(A_prev, W, b, activation):
      ...
      return A, cache
  def compute_cost(AL, Y):
      ...
      return cost
  def linear_activation_backward(dA, cache, activation):
      ...
      return dA_prev, dW, db
  def update_parameters(parameters, grads, learning_rate):
      ...
      return parameters
  ```

In [ ]:
### CONSTANTS DEFINING THE MODEL ####
n_x = 12288     # num_px * num_px * 3
n_h = 7
n_y = 1
layers_dims = (n_x, n_h, n_y)

In [ ]:
# GRADED FUNCTION: two_layer_model

def two_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    """
    Implements a two-layer neural network: LINEAR->RELU->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (n_x, number of examples)
    Y -- true "label" vector (containing 1 if cat, 0 if non-cat), of shape (1, number of examples)
    layers_dims -- dimensions of the layers (n_x, n_h, n_y)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- If set to True, this will print the cost every 100 iterations 
    
    Returns:
    parameters -- a dictionary containing W1, W2, b1, and b2
    """
    
    np.random.seed(1)
    grads = {}
    costs = []                              # to keep track of the cost
    m = X.shape[1]                           # number of examples
    (n_x, n_h, n_y) = layers_dims
    
    # Initialize parameters dictionary, by calling one of the functions you'd previously implemented
    ### START CODE HERE ### (≈ 1 line of code)
    parameters = None
    ### END CODE HERE ###
    
    # Get W1, b1, W2 and b2 from the dictionary parameters.
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # Forward propagation: LINEAR -> RELU -> LINEAR -> SIGMOID. Inputs: "X, W1, b1, W2, b2". Output: "A1, cache1, A2, cache2".
        ### START CODE HERE ### (≈ 2 lines of code)
        A1, cache1 = None
        A2, cache2 = None
        ### END CODE HERE ###
        
        # Compute cost
        ### START CODE HERE ### (≈ 1 line of code)
        cost = None
        ### END CODE HERE ###
        
        # Initializing backward propagation
        dA2 = - (np.divide(Y, A2) - np.divide(1 - Y, 1 - A2))
        
        # Backward propagation. Inputs: "dA2, cache2, cache1". Outputs: "dA1, dW2, db2; also dA0 (not used), dW1, db1".
        ### START CODE HERE ### (≈ 2 lines of code)
        dA1, dW2, db2 = None
        dA0, dW1, db1 = None
        ### END CODE HERE ###
        
        # Set grads['dWl'] to dW1, grads['db1'] to db1, grads['dW2'] to dW2, grads['db2'] to db2
        grads['dW1'] = dW1
        grads['db1'] = db1
        grads['dW2'] = dW2
        grads['db2'] = db2
        
        # Update parameters.
        ### START CODE HERE ### (approx. 1 line of code)
        parameters = None
        ### END CODE HERE ###

        # Retrieve W1, b1, W2, b2 from parameters
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if print_cost and i % 100 == 0:
            costs.append(cost)
       
    # plot the cost

    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

아래 코드 블록을 실행시켜서 파라미터를 학습해보세요. 모델이 실제로 동작하는지를 확인해봅시다. cost 값이 감소해야 합니다. 보통 2500회 반복하여 학습하는데 최대 5분정도의 시간이 걸립니다. 0회 반복의 비용이 아래의 모범 답안과 같은지 확인해보고, 일치하지 않는다면 이 주피터 노트북 상단의 사각형(⬛)을 클릭하여 실행을 중지하고 오류를 찾아보세요.

In [ ]:
parameters = two_layer_model(train_x, train_y, layers_dims = (n_x, n_h, n_y), num_iterations = 2500, print_cost=True)

**모범 답안**:
<table> 
    <tr>
        <td> <b>Cost after iteration 0</b></td>
        <td> 0.6930497356599888 </td>
    </tr>
    <tr>
        <td> <b>Cost after iteration 100</b></td>
        <td> 0.6464320953428849 </td>
    </tr>
    <tr>
        <td> <b>...</b></td>
        <td> ... </td>
    </tr>
    <tr>
        <td> <b>Cost after iteration 2400</b></td>
        <td> 0.048554785628770226 </td>
    </tr>
</table>

벡터화된 방식으로 구현했기 때문에, 훈련하는데 큰 시간이 걸리지 않았습니다. 만약 벡터화 방식이 아니었다면 10배는 많은 시간이 들었을 것입니다.

이제 훈련시킨 파라미터를 통해 이미지를 분류하는데 사용할 수 있습니다. 훈련 세트와 테스트 세트에 대한 예측 정확도를 확인해봅시다. 아래 코드를 실행시켜보세요.

In [ ]:
predictions_train = predict(train_x, train_y, parameters)

**모범 답안**:
<table> 
    <tr>
        <td><b>Accuracy</b></td>
        <td> 1.0 </td>
    </tr>
</table>

In [ ]:
predictions_test = predict(test_x, test_y, parameters)

**모범 답안**:
<table> 
    <tr>
        <td> <b>Accuracy</b></td>
        <td> 0.72 </td>
    </tr>
</table>

**참고** : 1500회 정도 되는 더 적은 반복 횟수로 모델을 실행하면 테스트 세트에서도 더 좋은 정확도를 얻을 수 있습니다. 이를 *early stopping* 이라고 하며, 다음 코스에서 이에 대해 설명합니다. *early stopping* 은 과적합을 방지하는 방법입니다.

어쨌거나 축하합니다! 2-layer 신경망이 70%의 정확도를 가졌던 로지스틱 회귀보다 더 나은 72%의 성능을 가지는 것으로 보입니다. $L$-layer 모델을 쓴다면 더 좋은 성능을 가지는지 살펴보겠습니다.

## 5. L-layer 신경망 구현하기 ##

**연습 문제**: 위와 마찬가지로 지난 과제에 구현한 helper 함수들을 사용해서 L-layer 신경망을 만들어보세요. 구조는 <u>[LINEAR -> RELU] $\times$(L-1) -> LINEAR -> SIGMOID</u> 와 같습니다. 지난번에 구현한 함수들의 대략적인 구조는 아래와 같습니다.
  ```python
  def initialize_parameters_deep(layers_dims):
    ...
    return parameters 
  def L_model_forward(X, parameters):
      ...
      return AL, caches
  def compute_cost(AL, Y):
      ...
      return cost
  def L_model_backward(AL, Y, caches):
      ...
      return grads
  def update_parameters(parameters, grads, learning_rate):
      ...
      return parameters
  ```

In [ ]:
### CONSTANTS ###
layers_dims = [12288, 20, 7, 5, 1] #  4-layer model

In [ ]:
# GRADED FUNCTION: L_layer_model

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- data, numpy array of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization. (≈ 1 line of code)
    ### START CODE HERE ###
    parameters = None
    ### END CODE HERE ###
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        ### START CODE HERE ### (≈ 1 line of code)
        AL, caches = None
        ### END CODE HERE ###
        
        # Compute cost.
        ### START CODE HERE ### (≈ 1 line of code)
        cost = None
        ### END CODE HERE ###
    
        # Backward propagation.
        ### START CODE HERE ### (≈ 1 line of code)
        grads = None
        ### END CODE HERE ###
 
        # Update parameters.
        ### START CODE HERE ### (≈ 1 line of code)
        parameters = None
        ### END CODE HERE ###
                
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

이제 4개의 층을 가지는 신경망을 학습시켜봅시다.

아래 코드 블록을 실행시켜서 파라미터를 학습해보세요. 모델이 실제로 동작하는지를 확인해봅시다. cost 값이 감소해야 합니다. 보통 2500회 반복하여 학습하는데 최대 5분정도의 시간이 걸립니다. 0회 반복의 비용이 아래의 모범 답안과 같은지 확인해보고, 일치하지 않는다면 이 주피터 노트북 상단의 사각형(⬛)을 클릭하여 실행을 중지하고 오류를 찾아보세요.

In [ ]:
parameters = L_layer_model(train_x, train_y, layers_dims, num_iterations = 2500, print_cost = True)

**모범 답안**:
<table> 
    <tr>
        <td><b>Cost after iteration 0</b></td>
        <td> 0.771749 </td>
    </tr>
    <tr>
        <td><b>Cost after iteration 100</b></td>
        <td> 0.672053 </td>
    </tr>
    <tr>
        <td><b>...</b></td>
        <td> ... </td>
    </tr>
    <tr>
        <td><b>Cost after iteration 2400</b></td>
        <td> 0.092878 </td>
    </tr>
</table>

In [ ]:
pred_test = predict(test_x, test_y, parameters)

<table>
    <tr>
    <td>
    <b>Train Accuracy</b>
    </td>
    <td>
    0.985645933014
    </td>
    </tr>
</table>

<table>
    <tr>
    <td>
    <b>Test Accuracy</b>
    </td>
    <td>
    0.985645933014
    </td>
    </tr>
</table>

동일한 테스트 세트에서 4개의 층을 가지는 신경망이 2-layer 신경망보다 8% 더 나은 정확도를 가집니다. 모델을 개선하는데 성공했습니다. 축하합니다.

심층 신경망을 어떻게 개선하는가에 관한 내용은 다음 코스에서 다를 예정입니다. 이 내용은 하이퍼 파라미터 (`learning_rate`, `layers_dims`, `num_iterations` 등등)를 체계적으로 검색하여 더 높은 정확도를 얻는 방법을 말합니다.

## 6. 결과 분석하기 ##

먼저 이 모델이 올바르지 않게 예측한 이미지를 확인해봅시다. 아래 코드를 실행하면, 모델이 라벨을 잘못 지정한 몇 가지 이미지들을 보여줍니다.

In [ ]:
print_mislabeled_images(classes, test_x, test_y, pred_test)

**모델이 제대로 수행하지 못하는 몇 가지 이미지 유형은 다음과 같습니다.**
- 특이한 위치의 고양이 몸
- 비슷한 색상의 배경에 고양이가 나타납니다.
- 특이한 고양이 색과 종
- 카메라 앵글
- 그림의 밝기
- 규모 변화 (고양이가 이미지에서 매우 크거나 작음)


## 7 - 자체 이미지를 통한 테스트 (선택 학습 / 채점 안됨) ##

과제를 모두 마치신 것을 축하드립니다! 이제 직접 자신이 가지고 있는 이미지를 가지고 모델의 output을 확인해봅시다. 아래의 방법을 따라해보세요.

1. 이 notebook 의 상단 표시줄에서 "File"을 클릭 후, "open"을 클릭하여 Cousera Hub로 이동합니다.
2. "images" 폴더에 있는 Jupyter Notebook의 디렉토리에 이미지를 추가합니다.
3. 아래 코드에서 이미지 이름을 변경합니다.
4. 코드를 실행하고, 알고리즘이 올바른 결과를 내는지 확인하세요! (1 = 고양이, 0 = 고양이 아님)

In [ ]:
## START CODE HERE ##
my_image = "my_image.jpg" # change this to the name of your image file 
my_label_y = [1] # the true class of your image (1 -> cat, 0 -> non-cat)
## END CODE HERE ##

fname = "images/" + my_image
image = np.array(ndimage.imread(fname, flatten=False))
my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((num_px*num_px*3,1))
my_image = my_image/255.
my_predicted_image = predict(my_image, my_label_y, parameters)

plt.imshow(image)
print ("y = " + str(np.squeeze(my_predicted_image)) + ", your L-layer model predicts a \"" + classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\" picture.")

**참고 문헌**:
- for auto-reloading external module: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython